In [48]:
import pandas as pd
import folium
import numpy as np
import json
import zipfile
import os
#import geopandas as gpd
import time
import datetime
import itertools
import requests
import bs4

In [55]:
countries = ['thailand', 'cambodia', 'laos', 'malaysia']

## UNZIP DATA

In [ ]:
for c in countries:
    path_to_zip_file = f'data/{c}.zip'
    directory_to_extract_to = 'data/'
    zip_ref = zipfile.ZipFile(path_to_zip_file, 'r')
    zip_ref.extractall(directory_to_extract_to)
    zip_ref.close()
    os.unlink(path_to_zip_file)

## READ JSON DATA

In [39]:
def json_reader(path):
    with open(path, 'r', encoding='utf-8') as fh:
        return json.loads(fh.read())

In [40]:
geojsons = {f: json_reader(f"data/{f.capitalize()}_AL4.GeoJson") for f in countries}

## GEOPANDAS FOR ADDITIONAL DATA

In [52]:
# concatenate data with country flags
for c in countries:
    gdf_ = gpd.GeoDataFrame.from_features(geojsons[c])
    gdf_['country'] = c
    try:
        gdf = gdf.append(gdf_, sort = False)
    except:
        gdf = gdf_

# drop some columns and calculate centroids
gdf = gdf[['geometry', 'name', 'localname', 'wikipedia', 'country']]
gdf['centr_lat'] = gdf.geometry.centroid.x
gdf['centr_lon'] = gdf.geometry.centroid.y
gdf = gdf.dropna()
gdf = gdf.drop_duplicates(subset=['name'])
gdf.head()

,geometry,name,localname,wikipedia,country,centr_lat,centr_lon
0,"(POLYGON ((104.4190919 15.6978084, 104.4190978...",Amnat Charoen Province,จังหวัดอำนาจเจริญ,en:Amnat Charoen Province,thailand,104.742211,15.896505
1,"(POLYGON ((100.1949768 14.5658321, 100.1960907...",Ang Thong Province,จังหวัดอ่างทอง,en:Ang Thong Province,thailand,100.355477,14.623284
2,"(POLYGON ((100.3278772 13.8041844, 100.3318503...",Bangkok,กรุงเทพมหานคร,en:Bangkok,thailand,100.623651,13.771980
3,"(POLYGON ((103.24334 18.357593, 103.2434 18.35...",Bueng Kan Province,จังหวัดบึงกาฬ,en:Bueng Kan Province,thailand,103.712064,18.148721
4,"(POLYGON ((102.4355316 14.8494415, 102.4369202...",Buri Ram Province,จังหวัดบุรีรัมย์,en:Buriram Province,thailand,102.962893,14.815251


In [53]:
gdf.shape

(136, 7)

## CREATE AREAS WE HAVE DATA ON

In [9]:
gdf['dis'] = [1, 1] + [f for f in range(2, 136)] 

In [10]:
gdf_dis = gdf.dissolveolve(by='dis')

In [11]:
gdf_dis.shape

(135, 6)

In [13]:
gdf_dis.head()

,geometry,name,localname,wikipedia,centr_lat,centr_lon
dis,,,,,,
1,"(POLYGON ((100.1949768 14.5658321, 100.1960907...",Amnat Charoen Province,จังหวัดอำนาจเจริญ,en:Amnat Charoen Province,104.742211,15.896505
2,"POLYGON ((100.3278772 13.8041844, 100.3318503 ...",Bangkok,กรุงเทพมหานคร,en:Bangkok,100.623651,13.771980
3,"POLYGON ((103.24334 18.357593, 103.2434 18.359...",Bueng Kan Province,จังหวัดบึงกาฬ,en:Bueng Kan Province,103.712064,18.148721
4,"POLYGON ((102.4355316 14.8494415, 102.4369202 ...",Buri Ram Province,จังหวัดบุรีรัมย์,en:Buriram Province,102.962893,14.815251
5,"POLYGON ((100.8489758 13.4791495, 100.849102 1...",Chachoengsao Province,จังหวัดฉะเชิงเทรา,en:Chachoengsao Province,101.451490,13.603437


In [59]:
## THAILAND
bangkok = ['Prachuap Khiri Khan Province', 'Phetchaburi Province', 'Ratchaburi Province', 'Samut Songkhram Province',
           'Nakhon Pathom Province', 'Samut Sakhon Province', 'Nonthaburi Province', 'Pathum Thani Province',
           'Bangkok', 'Samut Prakan Province', 'Nakhon Nayok Province', 'Prachin Buri Province', 'Sa Kaeo Province', 
           'Chachoengsao Province', 'Chon Buri Province']

rainycoast = ['Rayong Province', 'Chanthaburi Province', 'Trat Province']

peni_east = ['Prachuap Khiri Khan Province', 'Chumphon Province', 'Surat Thani Province', 'Nakhon Si Thammarat Province',
             'Phatthalung Province', 'Songkhla Province', 'Pattani Province', 'Yala Province', 'Narathiwat Province']

peni_west = ['Ranong Province', 'Phangnga Province', 'Phuket Province', 'Krabi Province', 'Trang Province',
             'Satun Province']

mountains = ['Tak Province', 'Mae Hong Son Province', 'Chiang Mai Province', 'Chiang Rai Province', 'Phayao Province',
             'Nan Province', 'Uttaradit Province', 'Phitsanulok Province', 'Loei Province']

all_oth_thai = bangkok + rainycoast + peni_east + peni_west + mountains
north = [f for f in gdf[gdf['country'] == 'thailand']['name'].tolist() if not f in all_oth_thai]

div_map = {f: 'Bangkok' for f in bangkok}


## GET WEATHER DATA

https://www.climatestotravel.com/climate/thailand

In [57]:
def get_country_weather(country):
    soup = soup_country_page(country)
    captions, dfs = scrape_tables(soup)
    return parse_tables_captions(captions, dfs)

def soup_country_page(country):
    r = requests.get(f'https://www.climatestotravel.com/climate/{country}')
    return bs4.BeautifulSoup(r.content, 'html5lib')

def scrape_tables(soup):
    tables = soup.find_all('table')
    dfs = []
    captions = []
    for table in tables:
        caption = table.find_all('caption')
        table_rows = table.find_all('tr')
        res = []
        for tr in table_rows:
            td = tr.find_all('td')
            row = [tr.text.strip() for tr in td if tr.text.strip()]
            if row:
                res.append(row)
        df = pd.DataFrame(res)
        dfs.append(df)
        captions.append(caption[0].text)
    
    assert len(captions) == len(dfs), 'Not matching number of captions and tables!'
    return captions, dfs

def parse_tables_captions(captions, dfs):
    new_dfs = {}
    for cap, df in zip(captions, dfs):
        if (('temper' in cap) & ('Sea' not in cap)):
            df.index = ['MinC', 'MaxC',  'MinF', 'MaxF']
            df.columns = [f for f in range(1,13)]
        elif 'precip' in cap:
            df.index = ['Prec(mm)', 'Prec(in)', 'Prec(days)']
            df.columns = [f for f in range(1,13)] + ['Year']
        elif 'Sunshin' in cap:
            df.index = ['HoursDaily']
            df.columns = [f for f in range(1,13)]
        else:
            continue
        new_dfs[cap] = df
    return new_dfs

In [58]:
weathers = [get_country_weather(c) for c in countries]

## CREATE MAP

In [30]:
m = folium.Map(location = (15.896505, 104.742211), zoom_start=8)

In [31]:
for k, v in geojsons.items():
    folium.GeoJson(
        v,
        name=k.capitalize()
    ).add_to(m)

fg = folium.FeatureGroup(name='Regions')
for name, lat, lon in zip(gdf['name'].tolist(), gdf['centr_lat'].tolist(), gdf['centr_lon'].tolist()):
    fg.add_child(folium.Marker(location=[lon, lat], popup=name))
    
m.add_child(fg)

folium.LayerControl().add_to(m)

In [32]:
m.save('outputs/map.html')